In [ ]:
import sys
import pandas
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec
import corner
from matplotlib.ticker import AutoMinorLocator
from matplotlib.ticker import MultipleLocator
from sklearn.linear_model import BayesianRidge

sys.path.append("../DependenceFitter-V2")

In [ ]:
from DependenceFitter import DependenceFitter1,DependenceFitter2
from DependenceFitter import median_and_error,get_computed_jpm, tau0_to_tau1
JPM, rho_iter = get_computed_jpm()

In [ ]:
df_burke = pandas.read_pickle("df_burke.pkl")
df_ren = pandas.read_pickle("df_ren.pkl")
df_combined = pandas.concat([df_burke,df_ren], axis=0)
df_combined = df_combined.reset_index(drop=True)

In [ ]:
def dependence_fit_thread(tau_out, Delta_AIC_low, Delta_AIC_hi, SNR, df, dependence_code, mask_code=0, nfit=2):    
    if mask_code == 0:
        mask_df = (df.Delta_AIC_low >1) & (df.Delta_AIC_hi >1) & (df.baseline/(df.ncadence-1) < 10**df.tau_mle)
        tau_out = tau_out[mask_df,:]
        Delta_AIC_low = Delta_AIC_low[mask_df,:]
        Delta_AIC_hi = Delta_AIC_hi[mask_df,:]
        SNR = SNR[mask_df,:]
        ncadence = df.ncadence.to_numpy()[mask_df]
        baseline = df.baseline.to_numpy()[mask_df]
        redshift_array = df.z.to_numpy()[mask_df]
        mask_array = ((Delta_AIC_low.T>1) & (Delta_AIC_hi.T>1) & (baseline/(ncadence-1) < 10**tau_out.T)).T
        choice = (np.sum(mask_array, axis=1) > 0)
    if mask_code == 1:
        mask_df = (df.Delta_AIC_low >1) & (df.Delta_AIC_hi >1) & (df.baseline/(df.ncadence-1) < 10**df.tau_mle) & (1e3*df.baseline > 10**df.tau_mle)
        tau_out = tau_out[mask_df,:]
        Delta_AIC_low = Delta_AIC_low[mask_df,:]
        Delta_AIC_hi = Delta_AIC_hi[mask_df,:]
        SNR = SNR[mask_df,:]
        ncadence = df.ncadence.to_numpy()[mask_df]
        baseline = df.baseline.to_numpy()[mask_df]
        redshift_array = df.z.to_numpy()[mask_df]
        mask_array = ((Delta_AIC_low.T>1) & (Delta_AIC_hi.T>1) & (baseline/(ncadence-1) < 10**tau_out.T)).T
        choice = (np.sum(mask_array, axis=1) > 0)
    if mask_code == 2:
        mask_df = (df.Delta_AIC_low >1) & (df.Delta_AIC_hi >1) &  (df.baseline/(df.ncadence-1) < 10**df.tau_mle) & (df.baseline > 10**df.tau_mle)
        tau_out = tau_out[mask_df,:]
        Delta_AIC_low = Delta_AIC_low[mask_df,:]
        Delta_AIC_hi = Delta_AIC_hi[mask_df,:]
        SNR = SNR[mask_df,:]
        ncadence = df.ncadence.to_numpy()[mask_df]
        baseline = df.baseline.to_numpy()[mask_df]
        redshift_array = df.z.to_numpy()[mask_df]
        mask_array = ((Delta_AIC_low.T>1) &(Delta_AIC_hi.T>1)& (baseline/(ncadence-1) < 10**tau_out.T) & (baseline > 10**tau_out.T)).T
        choice = (np.sum(mask_array, axis=1) > 0)
    if mask_code == 3:
        mask_df = (df.Delta_AIC_low >1) & (df.Delta_AIC_hi >1) &  (df.baseline/(df.ncadence-1) < 10**df.tau_mle) & (0.1*df.baseline > 10**df.tau_mle)
        tau_out = tau_out[mask_df,:]
        Delta_AIC_low = Delta_AIC_low[mask_df,:]
        Delta_AIC_hi = Delta_AIC_hi[mask_df,:]
        SNR = SNR[mask_df,:]
        ncadence = df.ncadence.to_numpy()[mask_df]
        baseline = df.baseline.to_numpy()[mask_df]
        redshift_array = df.z.to_numpy()[mask_df]
        mask_array = ((Delta_AIC_low.T>1) &(Delta_AIC_hi.T>1)& (baseline/(ncadence-1)  < 10**tau_out.T) & (0.1*baseline > 10**tau_out.T)).T
        choice = (np.sum(mask_array, axis=1) > 0)
    if mask_code == 4:
        mask_df = (df.Delta_AIC_low >1) & (df.Delta_AIC_hi >1) & (df.baseline/(df.ncadence-1) < 10**df.tau_mle) & (df.SNR_mle >1 )
        tau_out = tau_out[mask_df,:]
        Delta_AIC_low = Delta_AIC_low[mask_df,:]
        Delta_AIC_hi = Delta_AIC_hi[mask_df,:]
        SNR = SNR[mask_df,:]
        ncadence = df.ncadence.to_numpy()[mask_df]
        baseline = df.baseline.to_numpy()[mask_df]
        redshift_array = df.z.to_numpy()[mask_df]
        mask_array = ((Delta_AIC_low.T>1) &(Delta_AIC_hi.T>1)& (baseline/(ncadence-1)  < 10**tau_out.T) & (SNR.T>1)).T
        choice = (np.sum(mask_array, axis=1) > 0)
    if mask_code == 5:
        mask_df = (df.Delta_AIC_low >1) & (df.Delta_AIC_hi >1)
        tau_out = tau_out[mask_df,:]
        Delta_AIC_low = Delta_AIC_low[mask_df,:]
        Delta_AIC_hi = Delta_AIC_hi[mask_df,:]
        SNR = SNR[mask_df,:]
        ncadence = df.ncadence.to_numpy()[mask_df]
        baseline = df.baseline.to_numpy()[mask_df]
        redshift_array = df.z.to_numpy()[mask_df]
        mask_array = ((Delta_AIC_low.T>1) & (Delta_AIC_hi.T>1)).T
        choice = (np.sum(mask_array, axis=1) > 0)    
        
    if dependence_code == 0 :
        x = df.log_M_BH.to_numpy()[mask_df][choice]
        xerr = df.log_M_BH_e.to_numpy()[mask_df][choice]
        mask_int = 10**(0.5*x-1.7) < 0.1*baseline[choice]
    if dependence_code == 1 :
        x = df.log_M_BH.to_numpy()[mask_df][choice]
        xerr = df.log_M_BH_e.to_numpy()[mask_df][choice]
        mask_int = 10**(0.38*x-1.0) < 0.1*baseline[choice]
    if dependence_code == 2 :
        x = df.log_L_bol.to_numpy()[mask_df][choice]
        xerr = df.log_L_bol_e.to_numpy()[mask_df][choice]
        y = df.wl_rest.to_numpy()[mask_df][choice]
        yerr = 0.
        mask_int = 10**(0.72*x+ 1.19*y- 34.2) < 0.1*baseline[choice]    

    Coefs_tuple = []
    ind_array = np.array([np.array([np.random.choice(np.where(mask_array[choice][j,:])[0]) for j in range(sum(choice))]) for _ in range(nfit)])
    for i in range(nfit):
        tau_i = tau_out[choice][np.arange(sum(choice)), ind_array[i]]
        if dependence_code <=1:
            Fitter = DependenceFitter1(tau_i, baseline[choice], ncadence[choice], x, xerr, redshift=redshift_array[choice])
        if dependence_code ==2:
            Fitter = DependenceFitter2(tau_i, baseline[choice], ncadence[choice], x, y, xerr,yerr, redshift=redshift_array[choice])
        flat_samples = Fitter.fit()

        mask_good = 10**tau_i < 0.1*baseline[choice]

        if dependence_code <=1: 
            Coefs_tuple.append(np.array([
                *np.median(flat_samples, axis=0),
                *np.polyfit(x, (tau_i - np.log10(1+redshift_array[choice])), deg=1),
                *np.polyfit(x[mask_good], (tau_i - np.log10(1+redshift_array[choice]))[mask_good], deg=1),
                *np.polyfit(x[mask_int], (tau_i - np.log10(1+redshift_array[choice]))[mask_int], deg=1)]))

        if dependence_code ==2:
            model0 = BayesianRidge();
            model0.fit(np.column_stack((x,y)), tau_i- np.log10(1+redshift_array[choice]))
            model1 = BayesianRidge();
            model1.fit(np.column_stack((x[mask_good],y[mask_good] )), (tau_i- np.log10(1+redshift_array[choice]))[mask_good])
            model2 = BayesianRidge();
            model2.fit(np.column_stack((x[mask_int],y[mask_int])), (tau_i- np.log10(1+redshift_array[choice]))[mask_int])
            
            Coefs_tuple.append(np.array([
                *np.median(flat_samples, axis=0),
                *model0.coef_, model0.intercept_,
                *model1.coef_, model1.intercept_,
                *model2.coef_, model2.intercept_]))
            
    return Coefs_tuple

In [ ]:
df_mock = pandas.read_pickle("df_mock_25noise.pkl")
df_mock

In [ ]:
Coefs_Sum = []
for mask_code in [0,1,2,3,4,5]:
    for df_name in ["burke","ren","combined"]:
        df=eval(f"df_{df_name}")
        for dependence_code in range(3):
            df_imock = df_mock[(df_mock.df_name == df_name) & (df_mock.dependence_code == dependence_code)]
            tau_out = df_imock.iloc[0].tau_out
            Delta_AIC_low = df_imock.iloc[0].Delta_AIC_low
            Delta_AIC_hi = df_imock.iloc[0].Delta_AIC_hi
            SNR = df_imock.iloc[0].SNR
            Coefs_Sum.append(dependence_fit_thread(tau_out, Delta_AIC_low, Delta_AIC_hi, SNR, df=df, nfit=100, dependence_code=dependence_code, mask_code=mask_code))
    np.save("Coefs_Sum.npy", np.array(Coefs_Sum, dtype=object))

In [ ]:
# plt.hist(np.vstack(Coefs_Sum[0])[:,0])